In [130]:
import pandas as pd
import numpy as np
import re
import ast
import math

import warnings
warnings.filterwarnings("ignore")

In [131]:
pd.set_option("display.max_columns", None)

<h2 align="center" style="color:blue">Data Loading</h2>

In [132]:
flats = pd.read_csv("data/raw_data/hyderabad/flats.csv")
flats.head(3)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,Status,New/Resale,Price Negotiable,Age of Property,Bathrooms,Carpet area,Floor,Booking Amount,Balconies,Additional Rooms,Security Deposit,Security Deposit.1,Status.1,RERA ID,Price Negotiable.1,Age of Property.1,Facing,Balconies.1,New/Resale.1,Booking Amount.1,Additional Rooms.1,Overlooking,Ownership Type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/janapriya-eng...,Apartment,3 BHK Apartment,"2,120 sq ft",Janapriya Bahiti,Ameenpur,Hyderabad,1.44CrEMI,"6,800/ sq ft the price/ sq ft of this propert...",Under Construction,New,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P01100007344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['[]'],NaN
1,https://www.makaan.com/hyderabad/beams-and-col...,Apartment,2 BHK Apartment,"1,310 sq ft",Beams 4 Blocks,Kompally,Hyderabad,78.6LEMI,"6,000/ sq ft the price/ sq ft of this propert...",Ready to move,New,NaN,0 - 1 year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P02200001921,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Multipurpose Room, Rain Water Harvesting, J...",NaN
2,https://www.makaan.com/hyderabad/mahaveer-cons...,Apartment,4 BHK Apartment,"5,345 sq ft",Mahaveer Crystal Garden,Attapur,Hyderabad,4.54CrEMI,"8,500/ sq ft the price/ sq ft of this propert...",Under Construction,New,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P02500004287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Gymnasium, ATM, Lift(s), Jogging Track, Spo...",NaN


In [133]:
houses = pd.read_csv('data/raw_data/hyderabad/houses.csv')
houses.head(3)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,Status,New/Resale,Price Negotiable,Age of Property,Bathrooms,Carpet area,Floor,Booking Amount,Balconies,Additional Rooms,Security Deposit,Security Deposit.1,Status.1,RERA ID,Price Negotiable.1,Age of Property.1,Facing,Balconies.1,New/Resale.1,Booking Amount.1,Additional Rooms.1,Overlooking,Ownership Type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/sark-projects...,Villa,3 BHK Villa,"1,600 sq ft",Sark Townhomes Nadergul,Shankarpalli,Hyderabad,1.24CrEMI,"7,800/ sq ft the price/ sq ft of this propert...",Under Construction,New,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P02400007722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Intercom, Indoor Games, Car Parking, Club H...",NaN
1,https://www.makaan.com/hyderabad/vishal-projec...,Villa,4 BHK Villa,"2,925 sq ft",Vishal Vishal Sanjivini,Tukkuguda,Hyderabad,4.74CrEMI,"16,205/ sq ft the price/ sq ft of this proper...",Under Construction,New,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P02400005871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Rain Water Harvesting, Jogging Track, Car P...",NaN
2,https://www.makaan.com/hyderabad/aliens-group-...,Independent House,4 BHK Independent House,"3,942 sq ft",Aliens Space Station,Tellapur,Hyderabad,2.91CrEMI,"7,400/ sq ft the price/ sq ft of this propert...",Under Construction,New,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,Rera Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Indoor Games, ATM, Jogging Track, Lift(s), ...",NaN


In [134]:
flats.shape, houses.shape

((14750, 34), (4760, 34))

#### Merge the flats and houses dataframes

In [135]:
df = pd.concat([flats, houses], ignore_index=True)

In [136]:
df.sample(3)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,Status,New/Resale,Price Negotiable,Age of Property,Bathrooms,Carpet area,Floor,Booking Amount,Balconies,Additional Rooms,Security Deposit,Security Deposit.1,Status.1,RERA ID,Price Negotiable.1,Age of Property.1,Facing,Balconies.1,New/Resale.1,Booking Amount.1,Additional Rooms.1,Overlooking,Ownership Type,amenities,furnish_details
18834,https://www.makaan.com/hyderabad/builder-proje...,Villa,5 BHK Villa,"3,600 sq ft",NaN,Kardhanur,Hyderabad,1.9CrEMI,"5,277/ sq ft the price/ sq ft of this propert...",Under Construction,New,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,No,NaN,East,NaN,NaN,NaN,NaN,NaN,NaN,"['[Gymnasium, Lift(s), Intercom, Sports Facili...",NaN
248,https://www.makaan.com/hyderabad/candeur-devel...,Apartment,2 BHK Apartment,"1,165 sq ft",Candeur 40,Miyapur,Hyderabad,1.1CrEMI,"9,462/ sq ft the price/ sq ft of this propert...",Ready to move,New,NaN,0 - 1 year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P02400002183,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['[Shopping Mall, Lift(s), Indoor Games, Full ...",NaN
1921,https://www.makaan.com/hyderabad/builder-hmda-...,Apartment,2 BHK Apartment,"1,150 sq ft",NaN,Ameenpur,Hyderabad,57.48LEMINegotiable,"4,999/ sq ft the price/ sq ft of this propert...",Under Construction,NaN,NaN,NaN,2.0,NaN,3rd of 5,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,Yes,NaN,West,2.0,New,"2,00,000",NaN,Road View,Freehold,"['[Rain Water Harvesting, Intercom, Car Parkin...",NaN


##### Shuffle the data

In [137]:
df = df.sample(df.shape[0], ignore_index=True)
df.head(3)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,Status,New/Resale,Price Negotiable,Age of Property,Bathrooms,Carpet area,Floor,Booking Amount,Balconies,Additional Rooms,Security Deposit,Security Deposit.1,Status.1,RERA ID,Price Negotiable.1,Age of Property.1,Facing,Balconies.1,New/Resale.1,Booking Amount.1,Additional Rooms.1,Overlooking,Ownership Type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,"1,120 sq ft",NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,NaN,NaN,NaN,2.0,NaN,4th of 6,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,Yes,12 - 13 years,North,1.0,Resale,"5,00,000",1 rooms(pooja room),Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,"1,905 sq ft",ASBL ASBL Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,NaN,NaN,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,Yes,NaN,NaN,NaN,New,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,"14,400 sq ft",NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,NaN,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,No,NaN,North,NaN,NaN,NaN,NaN,NaN,NaN,['[]'],NaN


In [138]:
df.shape

(19510, 34)

##### Change the column names to lower case

In [139]:
df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.lower()
df.head(2)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,security_deposit.1,status.1,rera_id,price_negotiable.1,age_of_property.1,facing,balconies.1,new_resale.1,booking_amount.1,additional_rooms.1,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,"1,120 sq ft",NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,NaN,NaN,NaN,2.0,NaN,4th of 6,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,Yes,12 - 13 years,North,1.0,Resale,"5,00,000",1 rooms(pooja room),Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,"1,905 sq ft",ASBL ASBL Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,NaN,NaN,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,Yes,NaN,NaN,NaN,New,NaN,NaN,NaN,Freehold,['[]'],NaN


- In DataFrame, certain columns are repeated two times. So, Merge both columns.
- The repeated columns - new_resale, price_negotiable, age_of_property,booking_amount, balconies, additional_rooms, security_deposit

In [140]:
df.columns

Index(['link', 'property_type', 'name', 'area', 'society_name', 'locality',
       'city', 'price', 'price_per_sqft', 'status', 'new_resale',
       'price_negotiable', 'age_of_property', 'bathrooms', 'carpet_area',
       'floor', 'booking_amount', 'balconies', 'additional_rooms',
       'security_deposit', 'security_deposit.1', 'status.1', 'rera_id',
       'price_negotiable.1', 'age_of_property.1', 'facing', 'balconies.1',
       'new_resale.1', 'booking_amount.1', 'additional_rooms.1', 'overlooking',
       'ownership_type', 'amenities', 'furnish_details'],
      dtype='object')

In [141]:
# Function to merge repeated columns
def merge_and_drop(df, column):
    duplicate_column = column + '.1'
    
    if column in df.columns and duplicate_column in df.columns:
        # Merge columns using combine_first
        df[column] = df[column].combine_first(df[duplicate_column])
        # Drop the duplicate column
        df = df.drop(columns=[duplicate_column])
    
    return df

In [142]:
# List of columns with duplicates
repeated_columns = ['new_resale', 'price_negotiable', 'age_of_property', 
                    'booking_amount', 'balconies', 'additional_rooms', 'security_deposit']
# Apply the function to each repeated column
for col in repeated_columns:
    df = merge_and_drop(df, col)

df.head(3)

,link,property_type,name,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,"1,120 sq ft",NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,"1,905 sq ft",ASBL ASBL Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,"14,400 sq ft",NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


<h2 align="center" style="color:blue">Detailed Analysis</h2>

### 1. property_type

In [144]:
df['property_type'].unique()

array(['Apartment', 'Independent House', 'Villa', 'Independent Floor'],
      dtype=object)

### 2. name

In [145]:
df['name'].unique()

array(['2 BHK Apartment', '3 BHK Apartment', '10 BHK Independent House',
       '3 BHK Villa', '4 BHK Villa', '8 BHK Independent House',
       '4 BHK Apartment', '4 BHK Independent House',
       '3 BHK Independent House', '5 BHK Villa',
       '2 BHK Independent Floor', '6 BHK Independent House',
       '2 BHK Independent House', '5 BHK Apartment', '2 BHK Villa',
       '5 BHK Independent House', '6 BHK Apartment',
       '1 BHK Independent House', '9 BHK Independent House',
       '3 BHK Independent Floor', '6 BHK Independent Floor',
       '1 BHK Apartment', '5 BHK Independent Floor', '6 BHK Villa',
       '7 BHK Independent House', '12 BHK Independent House',
       '4 BHK Independent Floor', '7 BHK Apartment',
       '1 BHK Independent Floor', '11 BHK Independent House',
       '1 BHK Villa', '7 BHK Villa'], dtype=object)

##### From the name column we can extarct the bhk value.

In [146]:
df['name'].str.split(' ').str.get(0).astype('int')

0         2
1         3
2        10
3         2
4         3
         ..
19505     3
19506     3
19507     2
19508     4
19509     2
Name: name, Length: 19510, dtype: int64

In [147]:
df.insert(3, 'bhk', df['name'].str.split(' ').str.get(0).astype('int'))
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,"1,120 sq ft",NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,"1,905 sq ft",ASBL ASBL Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,"14,400 sq ft",NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 3. area

In [148]:
df['area'].unique()

array(['1,120 sq ft', '1,905 sq ft', '14,400 sq ft', ..., '954 sq ft',
       '510 sq ft', '6,105 sq ft'], dtype=object)

In [149]:
df['area'] = df['area'].str.split(' ').str.get(0).str.replace(',', '').astype('float')
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,ASBL ASBL Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 4. society_name

In [150]:
df['society_name'].unique()

array([nan, 'ASBL ASBL Spire', 'Prajay Virgin County Villas', ...,
       'Appam Venkatesh Sri Dharmasastra Enclave', 'AV Info Pride',
       'Vishwa Bobbys Rhythm'], dtype=object)

In [151]:
df['society_name'].isnull().sum()

np.int64(10563)

In [152]:
df['society_name'] = df['society_name'].str.strip().str.title()
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 5. locality

In [153]:
df['locality'].unique()

array(['Asmangarh', 'Kokapet', 'Banjara Hills', 'Kollur', 'Ameenpur',
       'Maheshwaram', 'Narsingi', 'Yousufguda', 'Tellapur', 'Suraram',
       'Hayathnagar', 'Nanakramguda', 'Vanasthalipuram', 'Bolarum',
       'Pragathi Nagar Kukatpally', 'Padmarao Nagar', 'Jubilee Hills',
       'Serilingampally', 'Nallagandla Gachibowli', 'Manikonda',
       'Gundlapochampally', 'Isnapur', 'Bachupally', 'Bowenpally',
       'Malkajgiri', 'Moosapet', 'Patancheru', 'Bundalguda',
       'Hitech City', 'nizampet road', 'Kukatpally', 'Lallaguda',
       'Kompally', 'Kondapur', 'Aminpur', 'Attapur', 'Chandanagar',
       'Puppalaguda', 'Kalasiguda', 'Adibatla', 'Peerzadiguda',
       'Himayat Nagar', 'Boduppal', 'Miyapur', 'Manchirevula', 'Velimela',
       'Satya Nagar Colony', 'Shadnagar', 'Mokila', 'Krishna Reddy Pet',
       'Gajulramaram Kukatpally', 'Dammaiguda', 'AS Rao Nagar',
       'Deluxe Colony', 'Sadashivpet', 'Yapral', 'LB Nagar', 'Beeramguda',
       'Osman Nagar', 'Annajiguda', 'Harsh

In [154]:
df['locality'] = df['locality'].str.strip().str.title()
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 6. price

In [155]:
df['price'].unique()

array(['58LEMINegotiable', '1.39CrEMINegotiable', '17.47CrEMI', ...,
       '70.1LEMI', '9.15CrEMI', '57.78LEMI'], dtype=object)

In [156]:
# Function to extract price and convert to lakhs
def clean_and_treat_price(price_str):
    if pd.isna(price_str):
        return None
    price_str = str(price_str)
    match = re.search(r'(\d+\.?\d*)(Cr|L)', price_str)
    if match:
        value = float(match.group(1))
        unit = match.group(2)

        # Convert to lakhs
        if unit == 'Cr':
            return value 
        elif unit == 'L':
            return round(value / 100, 2)
    return None

In [157]:
df['price'].apply(clean_and_treat_price)

0         0.58
1         1.39
2        17.47
3         0.45
4         0.77
         ...  
19505     0.93
19506     1.20
19507     0.32
19508     1.80
19509     1.60
Name: price, Length: 19510, dtype: float64

In [158]:
df.insert(8, 'price_in_crore', df['price'].apply(clean_and_treat_price))
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,"5,178/ sq ft the price/ sq ft of this propert...",Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,"7,349/ sq ft the price/ sq ft of this propert...",Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,17.47CrEMI,"12,131/ sq ft the price/ sq ft of this proper...",Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 7. price_per_sqft

In [159]:
df['price_per_sqft']

0         5,178/ sq ft the price/ sq ft of this propert...
1         7,349/ sq ft the price/ sq ft of this propert...
2         12,131/ sq ft the price/ sq ft of this proper...
3         3,401/ sq ft the price/ sq ft of this propert...
4         4,799/ sq ft the price/ sq ft of this propert...
                               ...                        
19505     5,799/ sq ft the price/ sq ft of this propert...
19506     6,451/ sq ft the price/ sq ft of this propert...
19507     3,742/ sq ft the price/ sq ft of this propert...
19508     5,921/ sq ft the price/ sq ft of this propert...
19509     12,800/ sq ft the price/ sq ft of this proper...
Name: price_per_sqft, Length: 19510, dtype: object

In [160]:
df['price_per_sqft'] = df['price_per_sqft'].str.split('/').str.get(0).str.replace(',','').astype('float')
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,12 - 13 years,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,7349.0,Under Construction,New,Yes,NaN,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,17.47CrEMI,12131.0,Ready to move,Resale,No,NaN,11.0,14400 sq ft,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN


### 8. age_of_property

In [161]:
df['age_of_property'].unique()

array(['12 - 13 years', nan, '5 - 6 years', '1 - 2 years',
       '10 - 11 years', '2 - 3 years', '9 - 10 years', '8 - 9 years',
       '6 - 7 years', '13 - 14 years', '0 - 1 year', '7 - 8 years',
       '4 - 5 years', '1 year', '14 - 15 years', '9 years', '7 years',
       '3 - 4 years', '6 years', '17 - 18 years', '11 - 12 years',
       '4 years', '13 years', '8 years', '22 - 23 years', '18 - 19 years',
       '20 - 21 years', '17 years', '14 years', '2 years',
       '16 - 17 years', '27 years', '11 years', '15 - 16 years',
       '19 years', '24 - 25 years', '31 years', '23 - 24 years',
       '3 years', '19 - 20 years', '124 - 125 years', '30 years',
       '5 years', '10 years', '50 - 51 years', '21 - 22 years',
       '28 - 29 years', '20 years', '18 years', '30 - 31 years',
       '25 - 26 years', '16 years', '37 - 38 years', '15 years',
       '12 years', '33 - 34 years'], dtype=object)

In [162]:
def categorize_age(age):
    if pd.isnull(age):
        return 'Unknown'
    if '-' in age:  
        years = list(map(int, age.replace(' years', '').replace(' year', '').split(' - ')))
        parsed_age =  math.ceil(sum(years) / len(years)) 
    elif 'year' in age:  
        parsed_age =  int(age.replace(' years', '').replace(' year', '').strip())  

    # Categorize the parsed age
    if parsed_age <= 5:
        return 'New Property'
    elif 6 <= parsed_age <= 10:
        return 'Moderately New Property'
    elif 11 <= parsed_age <= 50:
        return 'Old Property'
    else:
        return 'Very Old Property'

In [163]:
df['age_of_property'] = df['age_of_property'].apply(categorize_age)
df['age_of_property'].value_counts()

age_of_property
Unknown                    13636
New Property                3162
Moderately New Property     2034
Old Property                 674
Very Old Property              4
Name: count, dtype: int64

In [164]:
df.head(2)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4th of 6,"5,00,000",1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,7349.0,Under Construction,New,Yes,Unknown,3.0,NaN,9th of 36,NaN,NaN,NaN,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN


### 9. carpet_area

In [165]:
df['carpet_area'].unique()

array([nan, '14400 sq ft', '6000 sq ft', ..., '1858 sq ft', '1242 sq ft',
       '3870 sq ft'], dtype=object)

In [166]:
df['carpet_area'] = df['carpet_area'].str.split(' ').str.get(0).astype('float')
df[~df['carpet_area'].isnull()].head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,17.47CrEMI,12131.0,Ready to move,Resale,No,Unknown,11.0,14400.0,NaN,NaN,NaN,NaN,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,['[]'],NaN
8,https://www.makaan.com/hyderabad/builder-proje...,Independent House,8 BHK Independent House,8,6500.0,NaN,Yousufguda,Hyderabad,1.45,1.45CrEMINegotiable,2230.0,Ready to move,New,Yes,Old Property,8.0,6000.0,NaN,"5,00,000",4.0,1 rooms(servant room),No Deposit,Unfurnished,NaN,West,NaN,NaN,['[24 X 7 Security]'],"['[BED, Wifi]']"
9,https://www.makaan.com/hyderabad/my-home-const...,Apartment,3 BHK Apartment,3,1840.0,My Home Tridasa,Tellapur,Hyderabad,1.80,1.8CrEMI,9782.0,Ready to move,New,No,Unknown,3.0,1600.0,NaN,NaN,NaN,NaN,No Deposit,Unfurnished,P01100001916,West,NaN,NaN,"['[Gymnasium, Lift(s), Swimming Pool, Full Pow...",NaN


### 10. floor

In [167]:
df['floor'].unique()

array(['4th of 6', '9th of 36', nan, '1st of 5', '7th of 30', '4th of 5',
       '10th of 11', '2nd of 10', '2nd of 5', '2nd of 8', '1st of 3',
       '7th of 18', '3rd of 10', '13th of 20', '20th of 30', '3rd of 8',
       '2nd of 2', '2nd of 18', '17th of 27', '5th of 35', '16th of 35',
       '5th of 10', '14th of 14', '2nd of 7', '6th of 35', '34th of 35',
       '14th of 18', '5th of 13', '1st of 7', '10th of 21', '1st of 10',
       '1st of 8', '1st of 11', '7th of 13', '3rd of 5', '2nd of 6',
       '5th of 20', '7th of 14', '6th of 31', '7th of 35', '5th of 5',
       '12th of 24', '3rd of 3', '18th of 19', '9th of 10', '6th of 10',
       '6th of 19', '7th of 10', '19th of 49', '18th of 30', '7th of 19',
       '6th of 18', '10th of 25', '22nd of 25', '3rd of 20', '5th of 12',
       '11th of 16', '2nd of 14', '4th of 9', '11th of 32', '27th of 62',
       '3rd of 18', '6th of 15', '4th of 4', '3rd of 16', '2nd of 3',
       '8th of 18', '4th of 10', '2nd of 32', '5th of 8', '

In [168]:
def extract_floor_number(floor_str):
    if pd.isnull(floor_str):
        return None 

    # Check for "Gr" first
    if "Gr" in floor_str:
        return 0 # Represnt ground floor as 0

    # Use regex to find the first occurrence of a valid floor number
    match = re.match(r'(\d+)(?:st|nd|rd|th)?', floor_str)
    
    if match:
        return int(match.group(1))  # Extract the number before the suffix
    return None  # If no valid match found

In [169]:
df['floor'] = df['floor'].apply(extract_floor_number)
df['floor'].unique()

array([ 4.,  9., nan,  1.,  7., 10.,  2.,  3., 13., 20., 17.,  5., 16.,
       14.,  6., 34., 12., 18., 19., 22., 11., 27.,  8.,  0., 38., 24.,
       39., 23., 21., 15., 46., 43., 42., 28., 30., 48., 33., 26., 32.,
       40., 41., 36., 29., 31.])

### 11. booking_amount

In [170]:
df['booking_amount'].unique()

array([' 5,00,000', nan, ' 4,00,000', ' 10,00,000', ' 1,00,000',
       ' 2,00,000', ' 6,00,000', ' 9,00,000', ' 91,000', ' 87,600',
       ' 82,000', ' 1,80,000', ' 50,000', ' 86,000', ' 2,90,000',
       ' 68,000', ' 3,00,000', ' 3,66,800', ' 3,60,000', ' 1,28,888',
       ' 20,00,000', ' 81,000', ' 84,000', ' 86,500', ' 63,000',
       ' 89,000', ' 89,600', ' 3,65,800', ' 25,00,000', ' 10,000',
       ' 19,99,999', ' 76,000', ' 78,000', ' 7,40,000', ' 3,50,000',
       ' 56,000', ' 50,00,000', ' 67,000', ' 2,12,000', ' 35,000',
       ' 8,00,000', ' 79,000', ' 81,630', ' 96,000', ' 1,82,735',
       ' 17,00,000', ' 3,50,007', ' 68,700', ' 12,00,000', ' 62,000',
       ' 72,000', ' 1,56,000', ' 52,000', ' 68,880', ' 2,80,000',
       ' 74,000', ' 5', ' 15,00,000', ' 30,00,000', ' 9,24,370',
       ' 1,50,000', ' 64,000', ' 85,000', ' 73,000', ' 98,000',
       ' 7,00,000', ' 97,000', ' 97,860', ' 53,000', ' 3,69,800',
       ' 27,00,000', ' 57,000', ' 92,000', ' 95,000', ' 1,90,000',

In [171]:
df['booking_amount'] = df['booking_amount'].str.replace(',', '').astype('float')
df[~df['booking_amount'].isnull()].head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,1 rooms(pooja room),No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
4,https://www.makaan.com/hyderabad/builder-hmda-...,Apartment,3 BHK Apartment,3,1600.0,NaN,Ameenpur,Hyderabad,0.77,76.78LEMINegotiable,4799.0,Under Construction,New,Yes,Unknown,2.0,NaN,1.0,400000.0,2.0,1 rooms(pooja room),No Deposit,Unfurnished,NaN,East,Road View,NaN,['[]'],NaN
5,https://www.makaan.com/hyderabad/builder-abcfd...,Villa,3 BHK Villa,3,2000.0,NaN,Kollur,Hyderabad,1.30,1.3CrEMINegotiable,6500.0,Under Construction,New,Yes,Unknown,3.0,NaN,NaN,1000000.0,1.0,1 rooms(pooja room),No Deposit,Unfurnished,NaN,East,Garden View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']


### 12. 'additional_rooms'

In [172]:
df['additional_rooms'].isnull().sum()

np.int64(16027)

In [173]:
df['additional_rooms'].unique()

array(['1 rooms(pooja room)', nan, '1 rooms(servant room)',
       '3 rooms(study room, servant room, pooja room)',
       '1 rooms(study room)', '2 rooms(servant room, pooja room)',
       '2 rooms(study room, pooja room)',
       '2 rooms(study room, servant room)'], dtype=object)

In [174]:
# Function to clean the additional_rooms
pattern = re.compile(r'\((.*?)\)')
def extract_rooms(text):
    if isinstance(text, str):
        match = pattern.search(text)
        if match:
            return match.group(1)  # Extract room names inside parentheses
    return None

In [175]:
df['additional_rooms'] = df['additional_rooms'].apply(extract_rooms).str.strip()
df[~df['additional_rooms'].isnull()].head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,status.1,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,pooja room,No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
4,https://www.makaan.com/hyderabad/builder-hmda-...,Apartment,3 BHK Apartment,3,1600.0,NaN,Ameenpur,Hyderabad,0.77,76.78LEMINegotiable,4799.0,Under Construction,New,Yes,Unknown,2.0,NaN,1.0,400000.0,2.0,pooja room,No Deposit,Unfurnished,NaN,East,Road View,NaN,['[]'],NaN
5,https://www.makaan.com/hyderabad/builder-abcfd...,Villa,3 BHK Villa,3,2000.0,NaN,Kollur,Hyderabad,1.30,1.3CrEMINegotiable,6500.0,Under Construction,New,Yes,Unknown,3.0,NaN,NaN,1000000.0,1.0,pooja room,No Deposit,Unfurnished,NaN,East,Garden View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']


### 13. status.1

In [176]:
df['status.1'].unique()

array(['Semi-Furnished', 'Unfurnished', 'Furnished'], dtype=object)

##### The status.1 feature represents the furnishing_type. So, rename the column to furnishing_type

In [177]:
df.rename(columns={'status.1':'furnishing_type'}, inplace=True)
df.head(2)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,furnishing_type,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,pooja room,No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"['[Landscaped Gardens, Lift(s), Swimming Pool,...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,7349.0,Under Construction,New,Yes,Unknown,3.0,NaN,9.0,NaN,NaN,None,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,['[]'],NaN


### 14. amenities

In [178]:
df['amenities'][0]

"['[Landscaped Gardens, Lift(s), Swimming Pool, Gymnasium, Car Parking, 24 X 7 Security, Full Power Backup]']"

In [179]:
# Function to clean the amenities column and return as a comma-separated string
def clean_amenities(amenities_str):
    if pd.isnull(amenities_str):
        return ""
    
    # Remove the outer brackets and single quotes
    cleaned_str = amenities_str.strip("[]' ")
    
    # Find the innermost brackets and extract the string
    if cleaned_str.startswith('[') and cleaned_str.endswith(']'):
        cleaned_str = cleaned_str[1:-1]  # Remove outer brackets
    
    # Clean and concatenate the amenities
    amenities_string = ', '.join(amenity.strip() for amenity in cleaned_str.split(','))
    
    return amenities_string

In [180]:
df['amenities'] = df['amenities'].apply(clean_amenities)
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,furnishing_type,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,pooja room,No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",['[Sofa]']
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,7349.0,Under Construction,New,Yes,Unknown,3.0,NaN,9.0,NaN,NaN,None,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,,NaN
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,17.47CrEMI,12131.0,Ready to move,Resale,No,Unknown,11.0,14400.0,NaN,NaN,NaN,None,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,,NaN


### 15. furnish_details

In [181]:
df['furnish_details'][0]

"['[Sofa]']"

In [182]:
# Function to clean the amenities column and return as a comma-separated string
def clean_furnish_details(furnish_details_str):
    if pd.isnull(furnish_details_str):
        return ""
    
    # Remove the outer brackets and single quotes
    cleaned_str = furnish_details_str.strip("[]' ")
    
    # Find the innermost brackets and extract the string
    if cleaned_str.startswith('[') and cleaned_str.endswith(']'):
        cleaned_str = cleaned_str[1:-1]  # Remove outer brackets
    
    # Clean and concatenate the amenities
    furnish_details_string = ', '.join(amenity.strip() for amenity in cleaned_str.split(','))
    
    return furnish_details_string

In [183]:
df['furnish_details'] = df['furnish_details'].apply(clean_furnish_details)
df.head(3)

,link,property_type,name,bhk,area,society_name,locality,city,price_in_crore,price,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,furnishing_type,rera_id,facing,overlooking,ownership_type,amenities,furnish_details
0,https://www.makaan.com/hyderabad/builder-tgre-...,Apartment,2 BHK Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,58LEMINegotiable,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,pooja room,No Deposit,Semi-Furnished,NaN,North,Road View,Freehold,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa
1,https://www.makaan.com/hyderabad/asbl-asbl-spi...,Apartment,3 BHK Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,1.39CrEMINegotiable,7349.0,Under Construction,New,Yes,Unknown,3.0,NaN,9.0,NaN,NaN,None,No Deposit,Unfurnished,NaN,NaN,NaN,Freehold,,
2,https://www.makaan.com/hyderabad/builder-proje...,Independent House,10 BHK Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,17.47CrEMI,12131.0,Ready to move,Resale,No,Unknown,11.0,14400.0,NaN,NaN,NaN,None,No Deposit,Semi-Furnished,NaN,North,NaN,NaN,,


In [184]:
# Drop the columns which are not required further.
cols_to_drop = ['link', 'name', 'price', 'rera_id']
df1 = df.drop(cols_to_drop, axis='columns')
df1.head(3)

,property_type,bhk,area,society_name,locality,city,price_in_crore,price_per_sqft,status,new_resale,price_negotiable,age_of_property,bathrooms,carpet_area,floor,booking_amount,balconies,additional_rooms,security_deposit,furnishing_type,facing,overlooking,ownership_type,amenities,furnish_details
0,Apartment,2,1120.0,NaN,Asmangarh,Hyderabad,0.58,5178.0,Ready to move,Resale,Yes,Old Property,2.0,NaN,4.0,500000.0,1.0,pooja room,No Deposit,Semi-Furnished,North,Road View,Freehold,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa
1,Apartment,3,1905.0,Asbl Asbl Spire,Kokapet,Hyderabad,1.39,7349.0,Under Construction,New,Yes,Unknown,3.0,NaN,9.0,NaN,NaN,None,No Deposit,Unfurnished,NaN,NaN,Freehold,,
2,Independent House,10,14400.0,NaN,Banjara Hills,Hyderabad,17.47,12131.0,Ready to move,Resale,No,Unknown,11.0,14400.0,NaN,NaN,NaN,None,No Deposit,Semi-Furnished,North,NaN,NaN,,


In [185]:
df1.shape

(19510, 25)

In [186]:
df1.isnull().sum()

property_type           0
bhk                     0
area                    0
society_name        10563
locality                0
city                    0
price_in_crore        107
price_per_sqft        107
status                  3
new_resale              0
price_negotiable        0
age_of_property         0
bathrooms            2478
carpet_area         11051
floor               15031
booking_amount      13811
balconies           14812
additional_rooms    16027
security_deposit        0
furnishing_type         0
facing               7463
overlooking         15247
ownership_type      13010
amenities               0
furnish_details         0
dtype: int64

#### Save the cleaned df to csv

In [187]:
import os
os.makedirs('data/cleaned_data/hyderabad', exist_ok=True)

In [188]:
df1.to_csv('data/cleaned_data/hyderabad/hyd_properties_cleaned.csv', index=False)